# This notebook is to explore the census track data in geopandas

In [14]:
import geoplot as gplt
import geopandas as gpd
import geoplot.crs as gcrs
import imageio
import folium
import requests
import pathlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mapclassify as mc

%matplotlib inline
pd.set_option('display.max_colwidth', 999)

In [15]:
census_track = gpd.read_file('./data/census_track_2010_cook_county/tl_2010_17031_tract10.shp')
census_track.head()

,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,17,031,840300,17031840300,8403,Census Tract 8403,G5020,S,831224,0,+41.8320943,-087.6818822,"POLYGON ((-87.68608 41.82296, -87.68607 41.82302, -87.68608 41.82331, -87.68608 41.82338, -87.68612 41.82416, -87.68612 41.82419, -87.68613 41.82431, -87.68614 41.82435, -87.68620 41.82442, -87.68628 41.82450, -87.68636 41.82458, -87.68667 41.82489, -87.68663 41.82491, -87.68653 41.82497, -87.68650 41.82499, -87.68647 41.82501, -87.68644 41.82503, -87.68632 41.82510, -87.68625 41.82514, -87.68622 41.82516, -87.68619 41.82518, -87.68620 41.82525, -87.68621 41.82534, -87.68621 41.82547, -87.68621 41.82554, -87.68639 41.82559, -87.68644 41.82566, -87.68656 41.82583, -87.68668 41.82610, -87.68681 41.82635, -87.68682 41.82639, -87.68689 41.82651, -87.68694 41.82663, -87.68703 41.82683, -87.68704 41.82684, -87.68714 41.82709, -87.68724 41.82743, -87.68725 41.82746, -87.68729 41.82767, -87.68729 41.82782, -87.68730 41.82826, -87.68731 41.82841, -87.68731 41.82877, -87.68734 41.82987, -87.68735 41.83024, -87.68735 41.83042, -87.68735 41.83096, -87.68735 41.83115, -87.68736 41.83132, -87.6..."
1,17,031,840200,17031840200,8402,Census Tract 8402,G5020,S,831087,0,+41.8445748,-087.6491915,"POLYGON ((-87.64633 41.84009, -87.64632 41.83989, -87.64632 41.83986, -87.64631 41.83950, -87.64629 41.83841, -87.64628 41.83805, -87.64649 41.83804, -87.64693 41.83804, -87.64713 41.83804, -87.64734 41.83803, -87.64749 41.83803, -87.64771 41.83802, -87.64795 41.83802, -87.64810 41.83802, -87.64822 41.83802, -87.64858 41.83801, -87.64870 41.83801, -87.64879 41.83801, -87.64884 41.83801, -87.64927 41.83801, -87.64941 41.83800, -87.64983 41.83849, -87.65092 41.83976, -87.65109 41.83996, -87.65152 41.84046, -87.65169 41.84065, -87.65220 41.84125, -87.65237 41.84145, -87.65272 41.84186, -87.65377 41.84308, -87.65384 41.84316, -87.65413 41.84349, -87.65421 41.84360, -87.65443 41.84385, -87.65448 41.84390, -87.65449 41.84392, -87.65455 41.84399, -87.65456 41.84400, -87.65463 41.84408, -87.65463 41.84408, -87.65469 41.84415, -87.65484 41.84433, -87.65486 41.84434, -87.65492 41.84441, -87.65496 41.84446, -87.65507 41.84460, -87.65511 41.84464, -87.65569 41.84531, -87.65588 41.84554, -87.6..."
2,17,031,841100,17031841100,8411,Census Tract 8411,G5020,S,1143247,0,+41.8510058,-087.6350978,"POLYGON ((-87.62935 41.85280, -87.62934 41.85258, -87.62932 41.85200, -87.62931 41.85145, -87.62930 41.85107, -87.62930 41.85092, -87.62929 41.85046, -87.62928 41.85031, -87.62928 41.85009, -87.62927 41.84997, -87.62927 41.84949, -87.62927 41.84943, -87.62926 41.84921, -87.62925 41.84892, -87.62925 41.84889, -87.62923 41.84871, -87.62922 41.84807, -87.62922 41.84796, -87.62922 41.84783, -87.62921 41.84778, -87.62944 41.84762, -87.62938 41.84715, -87.62937 41.84706, -87.62938 41.84697, -87.62941 41.84676, -87.62935 41.84674, -87.62920 41.84667, -87.62915 41.84664, -87.62916 41.84663, -87.62918 41.84662, -87.62919 41.84658, -87.62919 41.84644, -87.62919 41.84613, -87.62917 41.84578, -87.62917 41.84556, -87.62932 41.84555, -87.62979 41.84555, -87.62979 41.84555, -87.62991 41.84554, -87.62995 41.84554, -87.62999 41.84555, -87.63035 41.84555, -87.63066 41.84554, -87.63093 41.84554, -87.63108 41.84554, -87.63157 41.84553, -87.63163 41.84553, -87.63166 41.84553, -87.63180 41.84553, -87.6..."
3,17,031,841200,17031841200,8412,Census Tract 8412,G5020,S,625405,0,+41.8555618,-087.6833420,"POLYGON ((-87.68814 41.85569, -87.68816 41.85649, -87.68817 41.85692, -87.68817 41.85704, -87.68817 41.85741, -87.68817 41.85754, -87.68817 41.85766, -87.68817 41.85803, -87.68817 41.85816, -87.68818 41.85839, -87.68821 41.85910, -87.68822 41.85934, -87.68807 41.85934, -87.68790 41.85934, -87.68774 41.85934, -87.68742 41.85934, -87.68631 41.85936, -87.68583 41.85937, -87.68510 41.85925, -87.68290 41.85888, -87.68217 41.85876, -87.68199 41.85876, -87.68146 41.85875, -87.68129 41.85875, -87.6807

In [16]:
census_track.shape

(1319, 13)

In [17]:
type(census_track['GEOID10'][0])

str

In [18]:
census_track['GEOID10'].isna().sum()

0

In [19]:
food_access = pd.read_csv('./data/food_access/FoodAccessResearchAtlasData2019.csv')
food_access.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,1001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,1001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,1001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,1001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0


In [20]:
type(food_access['CensusTract'][0])

numpy.int64

In [21]:
food_access['CensusTract'].isna().sum()

0

In [22]:
food_access['CensusTract'][0]

1001020100

In [23]:
# convert food_access['CensusTract'] to a string
food_access['CensusTract']= [str(ct_num) for ct_num in food_access['CensusTract']]

In [24]:
type(food_access['CensusTract'][0])

str

In [25]:
food_access['CensusTract'][0]

'1001020100'

In [26]:
food_access.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,1001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,1001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,1001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,1001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0


In [27]:
food_access[food_access['County'] == 'Cook County'].shape

(1320, 147)

### Merge census_tract and food_access on CensusTract and GEOID10 including only the data that exist in both files.

In [28]:

food_data = food_access.merge(census_track, left_on = 'CensusTract', right_on = 'GEOID10', how = 'inner')
food_data.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,17031010100,Illinois,Cook County,1,4854,2302,0,218.0,4.49,0,...,17031010100,101,Census Tract 101,G5020,S,381061,0,+42.0212617,-087.6698442,"POLYGON ((-87.66368 42.01940, -87.66384 42.01934, -87.66384 42.01929, -87.66389 42.01929, -87.66402 42.01929, -87.66443 42.01928, -87.66456 42.01927, -87.66486 42.01927, -87.66527 42.01926, -87.66574 42.01925, -87.66604 42.01925, -87.66648 42.01924, -87.66681 42.01924, -87.66780 42.01922, -87.66824 42.01921, -87.66824 42.01924, -87.66824 42.01933, -87.66824 42.01937, -87.66849 42.01936, -87.66887 42.01936, -87.66925 42.01936, -87.66950 42.01936, -87.66963 42.01937, -87.66975 42.01937, -87.67012 42.01937, -87.67049 42.01937, -87.67074 42.01937, -87.67086 42.01937, -87.67121 42.01937, -87.67133 42.01937, -87.67145 42.01937, -87.67180 42.01938, -87.67192 42.01938, -87.67215 42.01938, -87.67251 42.01938, -87.67287 42.01938, -87.67311 42.01939, -87.67336 42.01937, -87.67355 42.01964, -87.67406 42.02006, -87.67634 42.02193, -87.67710 42.02255, -87.67712 42.02263, -87.67718 42.02286, -87.67720 42.02294, -87.67628 42.02296, -87.67597 42.02297, -87.67351 42.02304, -87.67259 42.02306, -87.6..."
1,17031010201,Illinois,Cook County,1,6450,2463,0,163.0,2.53,0,...,17031010201,102.01,Census Tract 102.01,G5020,S,504470,0,+42.0160076,-087.6801485,"POLYGON ((-87.68009 42.01254, -87.68027 42.01254, -87.68075 42.01253, -87.68220 42.01250, -87.68234 42.01250, -87.68268 42.01250, -87.68275 42.01246, -87.68276 42.01246, -87.68300 42.01236, -87.68309 42.01232, -87.68313 42.01245, -87.68320 42.01267, -87.68357 42.01369, -87.68369 42.01403, -87.68377 42.01426, -87.68400 42.01493, -87.68408 42.01515, -87.68413 42.01532, -87.68416 42.01541, -87.68417 42.01546, -87.68424 42.01601, -87.68426 42.01622, -87.68430 42.01650, -87.68433 42.01679, -87.68443 42.01766, -87.68444 42.01776, -87.68446 42.01796, -87.68450 42.01826, -87.68457 42.01887, -87.68461 42.01918, -87.68464 42.01935, -87.68465 42.01948, -87.68432 42.01948, -87.68390 42.01948, -87.68331 42.01947, -87.68298 42.01947, -87.68281 42.01947, -87.68257 42.01946, -87.68187 42.01946, -87.68163 42.01946, -87.68139 42.01945, -87.68068 42.01945, -87.68045 42.01945, -87.68020 42.01944, -87.67974 42.01944, -87.67947 42.01944, -87.67923 42.01944, -87.67904 42.01944, -87.67883 42.01944, -87.6..."
2,17031010202,Illinois,Cook County,1,2818,1115,0,315.0,11.18,0,...,17031010202,102.02,Census Tract 102.02,G5020,S,351716,0,+42.0160503,-087.6733216,"POLYGON ((-87.67336 42.01937, -87.67311 42.01939, -87.67287 42.01938, -87.67251 42.01938, -87.67215 42.01938, -87.67192 42.01938, -87.67180 42.01938, -87.67145 42.01937, -87.67133 42.01937, -87.67134 42.01833, -87.67129 42.01827, -87.67111 42.01807, -87.67105 42.01803, -87.67085 42.01813, -87.67075 42.01817, -87.67046 42.01831, -87.67036 42.01836, -87.67036 42.01831, -87.67036 42.01818, -87.67036 42.01813, -87.67035 42.01792, -87.67034 42.01761, -87.67033 42.01727, -87.67032 42.01716, -87.67033 42.01706, -87.67033 42.01701, -87.67033 42.01698, -87.67033 42.01690, -87.67033 42.01687, -87.67033 42.01683, -87.67034 42.01675, -87.67033 42.01666, -87.67033 42.01652, -87.67032 42.01616, -87.67031 42.01599, -87.67030 42.01578, -87.67029 42.01547, -87.67028 42.01514, -87.67027 42.01493, -87.67027 42.01471, -87.67026 42.01438, -87.67024 42.01406, -87.67024 42.01384, -87.67023 42.01361, -87.67022 42.01330, -87.67021 42.01294, -87.67020 42.01271, -87.67063 42.01271, -87.67193 42.01268, -87.6..."
3,17031010300,Illinois,Cook County,1,6236,2826,0,791.0,12.68,0,...,17031010300,103,Census Tract 103,G5020,S,472017,0,+42.0159407,-087.6665351,"POLYGON ((-87.66506 42.01280, -87.66543 42.01279, -87.66592 42.01279, -87.66600 42.01279, -87.66614 42.01278, -87.66622 42.01278, -87.66622 42.01278, -87.66629 42.01278, -87.66637 42.01278, -87.66664 42.01277, -87.66768

In [29]:
food_data.shape

(1314, 160)

#### census tract has 1319 rows, food_access has 1320 rows. When merged food_data has 1314 rows.

In [30]:
food_data.isna().sum().sort_values(ascending = False)

lasnap20share    1314
laasian10        1314
lasnap10         1314
lahunv10share    1314
lahisp10share    1314
                 ... 
LATracts_half       0
LATracts1           0
LATracts10          0
LATracts20          0
geometry            0
Length: 160, dtype: int64

In [31]:
food_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Urban,1314.0,9.984779e-01,3.899886e-02,0.0,1.0,1.0,1.00,1.0
Pop2010,1314.0,3.953330e+03,1.788800e+03,237.0,2548.0,3848.0,5149.50,16735.0
OHU2010,1314.0,1.496466e+03,7.610063e+02,100.0,964.5,1424.0,1893.00,9751.0
GroupQuartersFlag,1314.0,3.805175e-03,6.159207e-02,0.0,0.0,0.0,0.00,1.0
NUMGQTRS,1314.0,6.870776e+01,3.525771e+02,0.0,0.0,2.0,30.75,10273.0
...,...,...,...,...,...,...,...,...
TractHispanic,1314.0,9.473075e+02,1.265664e+03,1.0,135.0,397.5,1181.50,6699.0
TractHUNV,1314.0,2.644140e+02,3.103921e+02,0.0,91.0,174.5,308.50,3473.0
TractSNAP,1314.0,2.258455e+02,1.915820e+02,0.0,77.0,185.0,315.00,1151.0
ALAND10,1314.0,1.845780e+06,3.416277e+06,8299.0,402537.5,869749.0,2072697.50,76741783.0


In [32]:
# drop the geometry column for now and then write data to csv
food_data.drop(columns = 'geometry').to_csv('./data/food_data.csv')